<a href="https://colab.research.google.com/github/lsGee/TIL/blob/master/01.Python/DS_%EC%8B%A4%EC%A0%84_Level_3_%EA%B0%95%EC%9D%98%EB%A1%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL

In [ ]:
import sqlite3
import pandas as pd

## Load Dataset

In [ ]:
## 해당 경로에 파일이 있으면 read 하고,
## 파일이 없으면 create 한다 (파일 새로 생성됨)
connection = sqlite3.connect("/content/drive/My Drive/Colab Notebooks/data/e-commerce.sqlite")

In [ ]:
query = "SELECT * FROM 'orders'"

order = pd.read_sql(query, connection, index_col="id")

order

,user_id,product_id,date,price,address,state
id,,,,,,
1,3,9,2017-01-01,500,Seoul,confirmed
2,1,7,2017-01-03,700,Seoul,confirmed
3,3,8,2017-01-03,900,Daejeon,confirmed
4,4,2,2017-01-07,500,nan,canceled
5,7,3,2017-01-09,700,Incheon,confirmed
6,5,7,2017-01-09,600,Busan,canceled
7,2,5,2017-01-10,200,nan,canceled
8,1,3,2017-02-04,600,Seoul,confirmed
9,4,8,2017-02-12,200,Daejeon,canceled


In [ ]:
## 가격대 높은순 상위 3개
query = """
  SELECT id, date, price, state 
  FROM 'orders' 
  ORDER BY price DESC
  LIMIT 3
"""

order = pd.read_sql(query, connection, index_col="id")

order

,date,price,state
id,,,
3,2017-01-03,900,confirmed
2,2017-01-03,700,confirmed
5,2017-01-09,700,confirmed


## 포함 조건

In [ ]:
## IN
query = "SELECT * FROM 'orders' WHERE address IN ('Seoul', 'Incheon')"

pd.read_sql(query, connection, index_col="id")

,user_id,product_id,date,price,address,state
id,,,,,,
1,3,9,2017-01-01,500,Seoul,confirmed
2,1,7,2017-01-03,700,Seoul,confirmed
5,7,3,2017-01-09,700,Incheon,confirmed
8,1,3,2017-02-04,600,Seoul,confirmed


In [ ]:
## 일부 문자(n) 포함
## 대소문자 구별 안 함
query = "SELECT * FROM 'orders' WHERE address LIKE '%n%'"

pd.read_sql(query, connection, index_col="id")

,user_id,product_id,date,price,address,state
id,,,,,,
3,3,8,2017-01-03,900,Daejeon,confirmed
4,4,2,2017-01-07,500,nan,canceled
5,7,3,2017-01-09,700,Incheon,confirmed
6,5,7,2017-01-09,600,Busan,canceled
7,2,5,2017-01-10,200,nan,canceled
9,4,8,2017-02-12,200,Daejeon,canceled
11,2,6,2017-02-28,420,Busan,confirmed


## 여러 개의 데이터(테이블) 하나로 합치기 : 쪼인~~
concat, merge보다 sql 을 바로 사용하는 것이 더 속도가 빠르다.

In [ ]:
query = """
  SELECT *
    FROM 'orders'
  LEFT JOIN 'users' ON 'orders'.user_id = 'users'.id
"""

pd.read_sql(query, connection)

,id,user_id,product_id,date,price,address,state,id,name,gender,age,email
0,1,3,9,2017-01-01,500,Seoul,confirmed,3.0,Park,male,37.0,park@dsschool.co.kr
1,2,1,7,2017-01-03,700,Seoul,confirmed,1.0,Kang,male,30.0,kang@gmail.com
2,3,3,8,2017-01-03,900,Daejeon,confirmed,3.0,Park,male,37.0,park@dsschool.co.kr
3,4,4,2,2017-01-07,500,nan,canceled,4.0,Lee,female,15.0,lee@empas.co.kr
4,5,7,3,2017-01-09,700,Incheon,confirmed,7.0,Choi,male,34.0,choi@yahoo.com
5,6,5,7,2017-01-09,600,Busan,canceled,5.0,Son,male,29.0,son@google.co.uk
6,7,2,5,2017-01-10,200,nan,canceled,2.0,Kim,female,22.0,kim@naver.com
7,8,1,3,2017-02-04,600,Seoul,confirmed,1.0,Kang,male,30.0,kang@gmail.com
8,9,4,8,2017-02-12,200,Daejeon,canceled,4.0,Lee,female,15.0,lee@empas.co.kr
9,10,8,2,2017-02-15,650,Daegu,confirmed,NaN,None,None,NaN,None


위 쿼리의 경우, JOIN 시 사용했던 id 컬럼이 중복되어 데이터프레임에 들어가게 된다.

따라서, JOIN 을 사용할 땐 아래와 같이 전체 컬럼을 SELECT 하지 않고 필요한 컬럼만 가져오는 게 좋다.

In [ ]:
query = """
  SELECT 'orders'.id, 'orders'.date, 'orders'.price, 'users'.name, 'users'.gender
  FROM 'orders'
  LEFT JOIN 'users' ON 'orders'.user_id = 'users'.id
"""

pd.read_sql(query, connection)  # 깔끔쓰

,id,date,price,name,gender
0,1,2017-01-01,500,Park,male
1,2,2017-01-03,700,Kang,male
2,3,2017-01-03,900,Park,male
3,4,2017-01-07,500,Lee,female
4,5,2017-01-09,700,Choi,male
5,6,2017-01-09,600,Son,male
6,7,2017-01-10,200,Kim,female
7,8,2017-02-04,600,Kang,male
8,9,2017-02-12,200,Lee,female
9,10,2017-02-15,650,None,None
